## install any packages here 

In [ ]:
import pandas as pd
import numpy as np

# utility
from sklearn.base import clone
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.utils.class_weight import compute_class_weight
from tabulate import tabulate

#viz
import seaborn as sns
import matplotlib.pyplot as plt
# from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

# parallel
import ray

try:
    ray.shutdown()
except:
    print("ray not started")

# ray.init()

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedShuffleSplit, LeaveOneOut, KFold
from scipy.stats import loguniform
from imblearn.under_sampling import RandomUnderSampler
# from mlxtend.feature_selection import SequentialFeatureSelector, ExhaustiveFeatureSelector
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.decomposition import PCA, NMF
from sklearn.compose import ColumnTransformer

import altair
import altair as alt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import copy
from skorch import NeuralNetClassifier
import torch.optim as optim
import optuna
from optuna.integration import OptunaSearchCV
# from mlxtend.feature_selection import SequentialFeatureSelector
# from tensorflow.keras.callbacks import EarlyStopping

# models
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet, RidgeClassifier, LinearRegression
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from lineartree import LinearTreeClassifier, LinearForestClassifier, LinearBoostClassifier


# analysis
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, classification_report

pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_columns", None) # show all cols

# from data_cleaning import clean_raw_data, create_dataset, get_all_results

from data_cleaning import clean_raw_data, create_dataset
from sklearn.ensemble import ExtraTreesClassifier
from mango import Tuner

RESULTS_DIR = "results/"
DATA_DIR = "data/"
SEED = 42

# reload modules in py files
%load_ext autoreload
%autoreload 2

In [ ]:
# ray.shutdown()

In [ ]:
# !pip install linear-tree

In [ ]:
# !pip uninstall --yes mlxtend

## clean dataset using functions in src/data_cleaning.py

In [ ]:
X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="previous_concussions")
print("No missing values in X: ", (X.isna().sum() == 0).all())
print("No missing values in y: ", (y.isna().sum() == 0).all())

print(X.columns)
display(X.head(5),X.shape ,y.head(5), y.shape)

print(y.value_counts())

In [ ]:
sampler = RandomUnderSampler(random_state=42, sampling_strategy="auto")
X_resampled, y_resampled = sampler.fit_resample(X, y)

print(X_resampled.shape, y_resampled.shape, y_resampled.value_counts())

## Model Tuning

1. Take best features from feature selection 
2. Tune model using best features 


In [ ]:
class NamedFeatureSelector(object):

    def __init__(self, column_names):
        all_col_names = ['age', 'DOB_d','weight','bimanual score: washer', "NHL",
                         'RT_V','RT_HR','Delta_RT','MT_V','MT_HR','Delta_MT','TMT_V','TMT_HR','CMT: V','CMT: HR','cvRT_V','cvRT_HR','stdRT_V','stdRT_HR','Ball Path_V','Ball Path_HR','FullPath_V','FullPath_HR','Delta_Fullpath','Corrective_V','Corrective_HR','PeakV_V','PeakV_HR','Delta_PV',
                         'AE_V','AE_HR','Delta_AE','VE_V','VE_HR','AbsOnAxis_HR','Delta_OnAxis','AbsOffAxis_V','AbsOffAxis_HR','Delta_OffAxis', 'AbsOnAxis_V'] 

        self.column_names = column_names
        self.column_idx = [all_col_names.index(col_name) for col_name in column_names]

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.iloc[:, self.column_idx]
            return X.to_numpy()
        else:
            X = X[:, self.column_idx]
            return X
    
    def fit(self, X, y=None):
        return self
    
        
    
    
    
features_sel = NamedFeatureSelector(["age", 'weight', "NHL"])
X, y = create_dataset(clean_raw_data("Brdi_db_march.xlsx"), target_col="previous_concussions")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.1, random_state=1, stratify=y)
display(X_train)
features_sel.transform(X_train)[:5]

#### Scale Certain Columns

In [ ]:
X, y_ = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="previous_concussions")

columns_to_exclude = ['DOB_d', 'age', 'weight', 'bimanualscore_washer', "NHL"]
cols_to_scale = np.setdiff1d(X.columns, columns_to_exclude).tolist()

# Define your column transformer
myStandardScaler = ColumnTransformer(
    remainder='passthrough',
    transformers=[
        ('StandardScaler', StandardScaler(), cols_to_scale),
    ],
    n_jobs=-1,  
)

myRobustScaler = ColumnTransformer(
    transformers=[
        ('RobustScaler', RobustScaler(), cols_to_scale),
    ],
    remainder='passthrough',  # Pass through any columns not specified in transformers
    n_jobs=-1,  # Use all available CPU cores for parallel processing
)

myMinMaxScaler = ColumnTransformer(
    transformers=[
        ('MinMaxScaler', MinMaxScaler(), cols_to_scale),
    ],
    remainder='passthrough',  # Pass through any columns not specified in transformers
    n_jobs=-1,  # Use all available CPU cores for parallel processing
)

pipeline = Pipeline([('scaler', myStandardScaler)])

X_transformed = pipeline.fit_transform(X)

# Get the names of the columns after scaling
# Get the names of the columns after scaling
X_scaled = pd.DataFrame(X_transformed, columns=[col.split("__")[1] for col in myStandardScaler.get_feature_names_out()])
display(X_scaled.head())
print("No missing values in X_scaled: ", (X_scaled.isna().sum() == 0).all())


#### Define Custom Optuna Distribution Class

Optuna requires that each parameter in the param_dist dict is of their distribution type. <br> We define a class that conforms to a distribution type in order to create distributions of objects/classes such as the scaling classes (ColumnTransformer) and undersampler (RandomUnderSampler)

In [ ]:
class CustomDistribution(optuna.distributions.CategoricalDistribution):
    def __init__(self, choices):
        self.choices = choices
        self.size = len(choices)

    def single(self):
        return self.choices[0]

    def to_external_repr(self, internal_repr):
        return self.choices[internal_repr]

    def to_internal_repr(self, external_repr):
        return self.choices.index(external_repr)

# ex
# scaler_dist = CustomDistribution([myStandardScaler])

#### Dimensionality Reduction using Principle component analysis

In [ ]:
X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="previous_concussions")
pca = PCA(n_components=.8, random_state=SEED)
pipe = Pipeline([('scaler', StandardScaler()), ('pca', pca)])
Xt = pipe.fit_transform(X)
plot = plt.scatter(Xt[:,0], Xt[:,1], c=y)
plt.legend(handles=plot.legend_elements()[0], labels=['No Concussion', 'Concussion'])
plt.show()

## function to get different types of cross validation

In [ ]:
def get_cross_validation(X, y, test_size=.2, n_splits=10, type="stratified", random_state=SEED):
    if type == "stratified":
        return list(StratifiedShuffleSplit(test_size=test_size, n_splits=n_splits, random_state=random_state).split(X, y))
    elif type == "leave_one_out":
        return list(LeaveOneOut().split(X, y))
    elif type=="K-Fold":
        return list(KFold(n_splits=n_splits, random_state=random_state, shuffle=True).split(X, y))

In [ ]:
def seaborn_conf_matrix(cm, model_name, result_dir="../figures/confusion_matricies"):
    plt.figure()
    plt.title(f"{model_name} Confusion Matrix")
    group_names = ["True Neg","False Pos","False Neg","True Pos"]
    group_counts = ["{0:0.0f}".format(value) for value in cm.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.savefig(f"{result_dir}/{model_name}_conf_matrix.png")

## Tuning Setup

In [ ]:
def graph_cv_results(results_df):
    plt.figure()
    # Plot loss scores using seaborn
    
    plt.xticks(rotation=90)
    sns.lineplot(data=results_df, x='params', y='mean_train_score', label='CV Training Score')
    sns.lineplot(data=results_df, x='params', y='mean_test_score', label='CV Test Score')
    plt.xlabel('')
    plt.xticks(ticks=[], labels=[])

In [ ]:
from sklearn.metrics import make_scorer

def scorer(*args):
    print(args)
    y_pred = clf.predict(X)
    score = f1_score(y, y_pred)
#     return score
    return 1


overfitting_scorer = make_scorer(scorer, greater_is_better=True)

In [ ]:
def create_classification_report_DF(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    report_df = pd.DataFrame(report).transpose()

    return report_df


create_classification_report_DF(list([0,0,0,0,0,0,0,0]), list([1,1,1,1,1,1,1,1]))

### Tune Model
Note: The decorator "@ray.remote" is useful to run different seeds in parallel using the ray python package

In [ ]:
scoring_dict = {

            "F1_unweighted" : "f1",
            "F1_weighted" : make_scorer(f1_score, average="weighted"),
            "F1_0" : make_scorer(f1_score, labels=[0], average=None),
            "F1_1" : make_scorer(f1_score, labels=[1], average=None),

            "Precision_unweighted" :  make_scorer(precision_score, zero_division=0),
            "Precision_weighted" : make_scorer(precision_score, zero_division=0, average="weighted"),
            "Precision_0" : make_scorer(precision_score,labels=[0], zero_division=0),
            "Precision_1" : make_scorer(precision_score,labels=[1], zero_division=0),

            "Recall_unweighted" : make_scorer(recall_score, zero_division=0),
            "Recall_weighted" : make_scorer(recall_score, zero_division=0, average="weighted"),
            "Recall_0" : make_scorer(recall_score, labels=[0], zero_division=0),
            "Recall_1" : make_scorer(recall_score, labels=[1], zero_division=0),
        }

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.tree import plot_tree
import graphviz
from sklearn.metrics import make_scorer
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

@ignore_warnings(category=ConvergenceWarning)
@ray.remote
def tune_model(model, param_dist, features=None, n_iter=200, target_col=None, random_state=SEED, scoring="F1_unweighted", search_type="random"):
    model_name = type(model).__name__
    if "Bagging" in model_name or "Ada" in model_name:
        model_name = model_name + "_" + model.estimator.__class__.__name__

    X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col=target_col)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=random_state, stratify=y)
  
    cv = get_cross_validation(X_train, y_train, n_splits=10, type="stratified", random_state=random_state)

    pipe = ImbPipeline([
            ('scaler', "passthrough"),
            ('selector', "passthrough"),
            ('model', model)
        ])

    print("SCORING:  ", scoring)
#    setup hyperparameter search using Sklearn random search 
    if search_type == "random":
        param_search = RandomizedSearchCV(pipe, 
                                            param_distributions=param_dist, 
                                            cv=cv,
                                            n_iter=n_iter,
                                            random_state=random_state,
                                            return_train_score=True,
                                            scoring=scoring_dict,
                                            n_jobs=-1,
                                            refit=scoring)
    elif search_type == "tpe":
        pipe = ImbPipeline([
            ('scaler', myStandardScaler),
            ('selector', "passthrough"),
            ('model', model)
        ])
        param_search = OptunaSearchCV(pipe, 
                                       param_distributions=param_dist,
                                       cv=cv,
                                       n_trials=n_iter,
                                       random_state=random_state,
                                       return_train_score=True,
                                       scoring=scoring_dict[scoring],
                                       n_jobs=-1,
                                       refit=True,
#                                        enable_pruning=True,
                                       timeout=600
                                           
        )

#   run randomsearch looking for best param configuration on training data
    param_search.fit(X_train, y_train)
    best_estimator = param_search.best_estimator_
#   Once hyperparameter search is complete, predict on validation data using best configuration

        
#     get training classification report
    y_pred_train = best_estimator.predict(X_train)
    train_report_df = create_classification_report_DF(y_train, y_pred_train)
#     print(f"Classification Report from training: {tabulate(train_report_df, headers='keys', tablefmt='psql')}")    
    
#     get validation classification report
    y_pred_val = best_estimator.predict(X_val)

    search_type_str = "Random" if search_type == 'random' else 'Tree-Parzen Estimator'
    print(f"Search Type: {search_type_str}")
    print(f"y val : {np.array(y_val)}")
    print(f"y pred: {y_pred_val}")
    weighted_f1 = f1_score(y_val, y_pred_val, average='weighted')
    print(f"Weighted F1: {weighted_f1}")
    
    val_report_df = create_classification_report_DF(y_val, y_pred_val)
#     print(f"Val classification report: {tabulate(val_report_df, headers='keys', tablefmt='psql')}")
    
#     collect estimator, params, and seed
    estimator_df = pd.DataFrame([{
                            "Best Estimator" : param_search.best_estimator_,
                            "Best Params" : param_search.best_params_, 
                            "Best Score" : param_search.best_score_,
                            "Search Type" : search_type_str,
                            "Seed" : random_state,
                            "Train F1 Weighted " : train_report_df.loc["weighted avg", "f1-score"],
                            "Test F1 Weighted " : val_report_df.loc["weighted avg", "f1-score"],
                            "Y Val" : np.array(y_val),
                            "Y Pred" : y_pred_val
                    }])
    
    
    return estimator_df, train_report_df, val_report_df
    

### Tune Model Sequential
Exact copy of tune_model, but without ray.remote() decorator, few modifications for mlp_torch

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
from sklearn.tree import plot_tree
import graphviz
from sklearn.metrics import make_scorer
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

@ignore_warnings(category=ConvergenceWarning)
def tune_model_sequential(model, param_dist, features=None, n_iter=200, target_col=None, random_state=SEED, scoring="F1_unweighted", search_type="random"):
    model_name = type(model).__name__
    if "Bagging" in model_name or "Ada" in model_name:
        model_name = model_name + "_" + model.estimator.__class__.__name__

    X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col=target_col)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=random_state, stratify=y)
  
    cv = get_cross_validation(X_train, y_train, n_splits=10, type="stratified", random_state=random_state)

    #     convert to tensors if pytorch model
    if "NeuralNet" in type(model).__name__:
        X_train = torch.FloatTensor(X_train.values).to(torch.float64)
        X_val = torch.FloatTensor(X_val.values).to(torch.float64)
        y_train = torch.FloatTensor(y_train.values).to(torch.float64)
        
    pipe = ImbPipeline([
            ('scaler', "passthrough"),
            ('selector', "passthrough"),
            ('model', model)
        ])

    print("SCORING:  ", scoring)
#    setup hyperparameter search using Sklearn random search 
    if search_type == "random":
        param_search = RandomizedSearchCV(pipe, 
                                            param_distributions=param_dist, 
                                            cv=cv,
                                            n_iter=n_iter,
                                            random_state=random_state,
                                            return_train_score=True,
                                            scoring=scoring_dict,
                                            n_jobs=1 if "NeuralNet" in type(model).__name__ else -1,
                                            refit=scoring)
    elif search_type == "tpe":
        pipe = ImbPipeline([
            ('scaler', myStandardScaler),
            ('selector', "passthrough"),
            ('model', model)
        ])
        param_search = OptunaSearchCV(pipe, 
                                       param_distributions=param_dist,
                                       cv=cv,
                                       n_trials=n_iter,
                                       random_state=random_state,
                                       return_train_score=True,
                                       scoring=scoring_dict[scoring],
                                       n_jobs=1 if "NeuralNet" in type(model).__name__ else -1,
                                       refit=True,
#                                        enable_pruning=True,
                                       timeout=600
                                           
        )

#   run randomsearch looking for best param configuration on training data
    param_search.fit(X_train, y_train)
    best_estimator = param_search.best_estimator_
#   Once hyperparameter search is complete, predict on validation data using best configuration

    
#     get training classification report
    y_pred_train = best_estimator.predict(X_train)
    train_report_df = create_classification_report_DF(y_train, y_pred_train)
#     print(f"Classification Report from training: {tabulate(train_report_df, headers='keys', tablefmt='psql')}")    
    
#     get validation classification report
    y_pred_val = best_estimator.predict(X_val)

    search_type_str = "Random" if search_type == 'random' else 'Tree-Parzen Estimator'
    print(f"Search Type: {search_type_str}")
    print(f"y val : {np.array(y_val)}")
    print(f"y pred: {y_pred_val}")
    weighted_f1 = f1_score(y_val, y_pred_val, average='weighted')
    print(f"Weighted F1: {weighted_f1}")
    
    val_report_df = create_classification_report_DF(y_val, y_pred_val)
#     print(f"Val classification report: {tabulate(val_report_df, headers='keys', tablefmt='psql')}")
    
#     collect estimator, params, and seed
    estimator_df = pd.DataFrame([{
                            "Best Estimator" : param_search.best_estimator_,
                            "Best Params" : param_search.best_params_, 
                            "Best Score" : param_search.best_score_,
                            "Search Type" : search_type_str,
                            "Seed" : random_state,
                            "Train F1 Weighted " : train_report_df.loc["weighted avg", "f1-score"],
                            "Test F1 Weighted " : val_report_df.loc["weighted avg", "f1-score"],
                            "Y Val" : np.array(y_val),
                            "Y Pred" : y_pred_val
                    }])
    
    
    return estimator_df, train_report_df, val_report_df
    

### Paralell Tune Loop

In [ ]:
from time import time

def outer_tune_loop(model, param_dist, n_iter=None,target_col="previous_concussions", elastic_net=False, scoring="F1_unweighted", parallel=True, search_type="random"):
    start = time()
    model_name = type(model).__name__
    if elastic_net == True:
        if "Bag" in model_name:
            model_name = "BaggingClassifier_ElasticNet"
        elif "Ada" in model_name:
            model_name = "AdaBoostClassifier_ElasticNet"
        else:
            model_name = "ElasticNet"
            
    elif "Bagging" in model_name or "Ada" in model_name:
        model_name = model_name + model.estimator.__class__.__name__
        
    model_name = model_name + "_" + search_type + "search"
        

    # random_states = [1, 10, 42, 69, 77, 11, 23, 99, 58, 91]
    random_states = [10, 69, 23, 42, 89]
#     random_states = [42, 89]
#     random_states = [42]

    estimator_dfs = pd.DataFrame()
    training_report_dfs = pd.DataFrame()
    val_report_dfs = pd.DataFrame()

    if parallel:
        print("Parallelizing with Ray")
        result_ids = [tune_model.remote(clone(model), param_dist, n_iter=n_iter, target_col=target_col, random_state=random_states[i], scoring=scoring, search_type=search_type) for i in range(len(random_states))]
        print(f"len result ids: {len(result_ids)}")

        for estimator_df, training_report_df, val_report_df in ray.get(result_ids):

    #         collect each of the three dfs for one seed
            estimator_dfs = pd.concat([estimator_dfs, estimator_df])
            training_report_dfs = pd.concat([training_report_dfs, training_report_df])
            val_report_dfs = pd.concat([val_report_dfs, val_report_df])  
          
    else:
        print("Running Sequentially")
        for seed in random_states:
            estimator_df, training_report_df, val_report_df = tune_model_sequential(clone(model), param_dist, n_iter=n_iter, target_col=target_col, random_state=seed, scoring=scoring, search_type=search_type)

        #         collect each of the three dfs for one seed
            estimator_dfs = pd.concat([estimator_dfs, estimator_df])
            training_report_dfs = pd.concat([training_report_dfs, training_report_df])
            val_report_dfs = pd.concat([val_report_dfs, val_report_df])    


    
#     val_f1 = round(outer_loop_results['Validation F1'].mean(), 3)
#     val_precision = round(outer_loop_results['Validation Precision'].mean(), 3)
#     val_recall = round(outer_loop_results['Validation Recall'].mean(), 3)

#     print(f"Mean Validation Score Across {len(random_states)} trials: {val_f1}")
#     print(f"Mean Validation Precision Across {len(random_states)} trials: {val_precision}")
#     print(f"Mean Validation Recall Across {len(random_states)} trials: {val_recall}")
   
        
#     outer_loop_results.to_excel(f"../{RESULTS_DIR}{model_name}.xlsx", index=False)
#     print(f"Training time for {len(random_states)} seeds: {round(time() - start, 3)}")
#     return outer_loop_results

    estimator_dfs.to_excel(f"../{RESULTS_DIR}{model_name}-estimator_df.xlsx", index=True)
    training_report_dfs.to_excel(f"../{RESULTS_DIR}{model_name}-train_df.xlsx", index=True)
    val_report_dfs.to_excel(f"../{RESULTS_DIR}{model_name}-val_df.xlsx", index=True)
    
    return estimator_dfs, training_report_dfs.groupby(training_report_dfs.index).mean(), val_report_dfs.groupby(val_report_dfs.index).mean()

## Decision Tree

### TPE Decision Tree

In [ ]:
model = DecisionTreeClassifier(random_state=SEED)

param_dist = {
        'scaler' : CustomDistribution([myStandardScaler]),
        "selector" : CustomDistribution([RandomUnderSampler(random_state=42)]),
        "model__criterion" : optuna.distributions.CategoricalDistribution(["gini"]),
        "model__max_depth" : optuna.distributions.IntDistribution(2, 8),
        "model__min_samples_split" : optuna.distributions.IntDistribution(8, 15),
        "model__min_samples_leaf" : optuna.distributions.IntDistribution(8, 15),
        "model__max_features" : optuna.distributions.CategoricalDistribution(["sqrt"]),
        "model__class_weight" : optuna.distributions.CategoricalDistribution(["balanced"]),
    }

dt_est_tpe, dt_train_tpe, dt_val_tpe = outer_tune_loop(model=model, param_dist=param_dist,scoring="F1_weighted" ,n_iter=100, target_col="previous_concussions", parallel=False, search_type="tpe")

#### TPE Decision Tree Results

In [ ]:
display(dt_train_tpe, dt_val_tpe)

In [ ]:
model = DecisionTreeClassifier(random_state=SEED)

param_dist = {
        'scaler' : [myStandardScaler],
        "selector" : [
#             None
            RandomUnderSampler(random_state=42),
#             SelectFromModel(LinearSVC(random_state=SEED, penalty="l1", dual=False)),
#             SelectFromModel(ExtraTreesClassifier(random_state=SEED)), 
            NamedFeatureSelector(["Delta_MT", "Delta_AE", "cvRT_HR", "Corrective_HR", "RT_V", "age", "NHL"]),
        ],
        "model__criterion" : ["gini"],
        "model__max_depth" : [4, 5],
        "model__min_samples_split" : [13],
        "model__min_samples_leaf" : [11],
        "model__max_features" : ["sqrt"],
        "model__class_weight" : ["balanced"],
    }

dt_est, dt_train, dt_val = outer_tune_loop(model=model, param_dist=param_dist,scoring="F1_weighted" ,n_iter=100,target_col="previous_concussions", parallel=True)

#### Decision Tree Results

In [ ]:
display(dt_train, dt_val)

In [ ]:
model = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=SEED), random_state=SEED)
param_dist = {
        "selector" : [
            RandomUnderSampler(random_state=42),
#             None,
#             SelectFromModel(LinearSVC(random_state=SEED, penalty="l1", dual=False)),
#             SelectFromModel(ExtraTreesClassifier(random_state=SEED)), 
            NamedFeatureSelector(["Delta_MT", "Delta_AE", "cvRT_HR", "Corrective_HR", "RT_V", "age", "NHL"]),
        ],
        'scaler' : [myMinMaxScaler, myStandardScaler],
        "model__n_estimators" : [5, 10, 50, 100, 200],
        "model__estimator__criterion" : ["entropy"],
        "model__estimator__max_depth" : range(1, 5),
        "model__estimator__min_samples_split" : [6],
        "model__estimator__min_samples_leaf" : [4],
        "model__estimator__max_features" : ["sqrt"],
        "model__estimator__class_weight" : ["balanced"]
    }


bagged_dt_est, bagged_dt_train, bagged_dt_val = outer_tune_loop(model=model, param_dist=param_dist,scoring="F1_weighted" ,n_iter=250,target_col="previous_concussions")


#### Bagged Decision Tree Results

In [ ]:
display(bagged_dt_est, bagged_dt_train, bagged_dt_val)

In [ ]:
model = AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=SEED), random_state=SEED)
param_dist = {
        "selector" : [
            RandomUnderSampler(random_state=42)
#             None,
#             SelectFromModel(LinearSVC(random_state=SEED, penalty="l1", dual=False)),
#             SelectFromModel(ExtraTreesClassifier(random_state=SEED)), 
#             NamedFeatureSelector(["Delta_MT", "Delta_AE", "cvRT_HR", "Corrective_HR", "RT_V", "age"]),
        ],
        'scaler' : [myStandardScaler, myMinMaxScaler, myRobustScaler, None],
        "model__n_estimators" : [5, 10, 50],
        "model__estimator__criterion" : ["gini", "entropy"],
        "model__estimator__max_depth" : range(1, 7),
        "model__estimator__min_samples_split" : range(2, 7),
        "model__estimator__min_samples_leaf" : range(1, 7),
        "model__estimator__max_features" : ["sqrt", "log2", None],
        "model__estimator__class_weight" : ["balanced"]
    }


boosted_dt_est, boosted_dt_train, boosted_dt_val  = outer_tune_loop(model=model, param_dist=param_dist, n_iter=100,target_col="previous_concussions")

#### Boosted Decision Tree Results

In [ ]:
display(boosted_dt_est, boosted_dt_train, boosted_dt_val)

-----
## Collect Results

In [ ]:
import os 
from scipy import stats
import pandas as pd
from tabulate import tabulate


# ----- Random Search -----
# weak classifier results
estimator_results = pd.DataFrame()
train_results = pd.DataFrame()
val_results = pd.DataFrame()

# ensemble results
ensemble_estimator_results = pd.DataFrame()
ensemble_train_results = pd.DataFrame()
ensemble_val_results = pd.DataFrame()

# ----- Tree-Structured Parzen Estimator Search -----
tpe_estimator_results = pd.DataFrame()
tpesearch_train_results = pd.DataFrame()
tpesearch_val_results = pd.DataFrame()

for file_name in os.listdir("../results"):
    if "xlsx" in file_name:
        df = pd.read_excel(f"../results/{file_name}", index_col=0)
        df["model"] = file_name.split("_")[0]
        df['model'] = df['model'].str.replace('-train', '').str.replace('-val', '')
        df = df.set_index("model", append=True).swaplevel(0, 1, axis=0)
        df.index = df.index.set_names(["model", "metric type"])
        df = df.rename(index={'0': 'No Prior Concussion: 0'})
        df = df.rename(index={'1': 'Prior Concussion: 1'})
        df = df.rename(index={"0.0": 'No Prior Concussion: 0'})
        df = df.rename(index={"1.0": 'Prior Concussion: 1'})
#         df.rename_axis(index=['model', 'metric'])
#         df.index=[os.path.splitext(file_name)[0]] * len(df)

#         df["Validation F1 STD"] = df["Validation F1"].std()
        # df = df[df["Validation Z Score"] < 1.7]
        if "estimator" in file_name:
            if "tpesearch" in file_name:
                tpe_estimator_results = pd.concat([tpe_estimator_results, df])
            elif "Bagging" in file_name or "Ada" in file_name:
                ensemble_estimator_results = pd.concat([ensemble_estimator_results, df])
            elif 'randomsearch' in file_name:
                estimator_results = pd.concat([estimator_results, df])
                
        elif "train" in file_name:
            
            if "tpesearch" in file_name:
              
                tpesearch_train_results = pd.concat([tpesearch_train_results, df])   
            elif "Bagging" in file_name or "Ada" in file_name:
                ensemble_train_results = pd.concat([ensemble_train_results, df])
            elif 'randomsearch' in file_name:
                train_results = pd.concat([train_results, df])
                
        elif "val" in file_name:
            
            if "tpesearch" in file_name:
                tpesearch_val_results = pd.concat([tpesearch_val_results, df])
            elif "Bagging" in file_name or "Ada" in file_name:
                ensemble_val_results = pd.concat([ensemble_val_results, df])
            elif 'randomsearch' in file_name:
                print(file_name)
                val_results = pd.concat([val_results, df])
                

## Avg Model and Ensemble Results

In [ ]:
def sort_by_metric(df, metric_type="weighted avg", metric="f1-score"):
#     temp = df

#     filter df to get order of (metric_type, metric)
    temp = df.reset_index()
    temp = temp[temp["metric type"] == metric_type].sort_values(metric, ascending=False)
    model_indices = {model: index for index, model in enumerate(temp['model'])}

    
#     set a new temp column that has ordering 
    df = df.reset_index("model")
    for key, value in model_indices.items():
        df.loc[df["model"] == key,  "test index"] = value

#         sort df on temp column and reset df to how it was
    df = df.reset_index().set_index(["model", "metric type"]).sort_values(["test index", "metric type"]).drop(columns=["test index"])
    
#     def sort_group(group):
#         return group.sort_values(by='metric type')

    
#     df.index = df.index.sortlevel(1, ascending=False)
#     df.sort_index(level='metric type', sort_remaining=False, inplace=True)
    return df


In [ ]:
METRIC_TYPE = "weighted avg"
METRIC = "f1-score"

#### AVG TPE Train Results

In [ ]:
avg_tpe_train_results = tpesearch_train_results.groupby(tpesearch_train_results.index.names, group_keys=False).mean()
avg_tpe_train_results = sort_by_metric(avg_tpe_train_results, METRIC_TYPE, METRIC)
avg_tpe_train_results.to_excel("../mean_results/avg_tpe_train_results_80_20.xlsx", index=True)
avg_tpe_train_results

#### AVG TPE Val Results

In [ ]:
avg_tpe_val_results = tpesearch_val_results.groupby(tpesearch_val_results.index.names, group_keys=False).mean()
avg_tpe_val_results = sort_by_metric(avg_tpe_val_results, METRIC_TYPE, METRIC)
avg_tpe_val_results.to_excel("../mean_results/avg_tpe_train_results_80_20.xlsx", index=True)
avg_tpe_val_results

#### AVG TPE Results

In [ ]:


avg_tpe_results = pd.concat([avg_tpe_train_results, avg_tpe_val_results], axis=1, keys=["train", 'generalize'])
# weighted_avg_rows = pd.IndexSlice[avg_tpe_results.index.get_level_values(1) == "weighted avg"]
# and apply styling to it via the `subset` arg; first arg is styler function above
# avg_tpe_results = avg_tpe_results.style.applymap(df_style, subset=weighted_avg_rows)
avg_tpe_results

#### AVG Train Results

In [ ]:
avg_train_results = train_results.groupby(train_results.index.names, group_keys=False).mean()
avg_train_results = sort_by_metric(avg_train_results, METRIC_TYPE, METRIC)
avg_train_results.to_excel("../mean_results/avg_train_results_80_20.xlsx", index=True)
avg_train_results

#### AVG Val Results

In [ ]:
avg_val_results = val_results.groupby(val_results.index.names, group_keys=False).mean()
avg_val_results = sort_by_metric(avg_val_results, METRIC_TYPE, METRIC)
avg_val_results.to_excel("../mean_results/avg_val_results_80_20.xlsx", index=True)
avg_val_results

#### AVG Results

In [ ]:
avg_results = pd.concat([avg_train_results, avg_val_results], axis=1, keys=["train", 'generalize'])
avg_results

#### Random Search Paper Table

In [ ]:
paper_avg_rs_results_df = avg_results.loc[(slice(None), 'weighted avg'), :].droplevel('metric type').drop('support', axis = 1, level = 1)
paper_avg_rs_results_df.loc["Mean"] = paper_avg_rs_results_df.mean()
paper_avg_rs_results_df = paper_avg_rs_results_df.round(3)
display(paper_avg_rs_results_df)
print(paper_avg_rs_results_df.to_latex())

#### TPE Search Paper Table 

In [ ]:
paper_avg_tpe_results_df = avg_tpe_results.loc[(slice(None), 'weighted avg'), :].droplevel('metric type').drop('support', axis = 1, level = 1)
paper_avg_tpe_results_df.loc["Mean"] = paper_avg_tpe_results_df.mean()
paper_avg_tpe_results_df = paper_avg_tpe_results_df.round(3)
display(paper_avg_tpe_results_df)
print(paper_avg_tpe_results_df.to_latex())

#### AVG Ensemble Train Results

In [ ]:
avg_ensemble_train_results = ensemble_train_results.groupby(train_results.index.names, group_keys=False).mean()
avg_ensemble_train_results = sort_by_metric(avg_ensemble_train_results, METRIC_TYPE, METRIC)
avg_ensemble_train_results.to_excel("../mean_results/avg_ensemble_train_results_80_20.xlsx", index=True)
avg_ensemble_train_results

#### AVG Ensemble Val Results

In [ ]:
avg_ensemble_val_results = ensemble_val_results.groupby(train_results.index.names, group_keys=False).mean()
avg_ensemble_val_results = sort_by_metric(avg_ensemble_val_results, METRIC_TYPE, METRIC)
avg_ensemble_val_results.to_excel("../mean_results/avg_ensemble_val_results_80_20.xlsx", index=True)
avg_ensemble_val_results

#### AVG Ensemble Results

In [ ]:
avg_ensemble_results = pd.concat([avg_ensemble_train_results, avg_ensemble_val_results], axis=1, keys=["train", 'val'])
avg_ensemble_results

#### Ensemble Results Paper Table 

In [ ]:
avg_ensemble_results

paper_avg_ensenble_results = avg_ensemble_results.loc[(slice(None), 'weighted avg'), :].droplevel('metric type').drop('support', axis = 1, level = 1)
paper_avg_ensenble_results.loc["Mean"] = paper_avg_ensenble_results.mean()
paper_avg_ensenble_results = paper_avg_ensenble_results.round(3)
display(paper_avg_ensenble_results)
print(paper_avg_ensenble_results.to_latex())

#### Random Search Explainable Vs BlackBox Random Search Paper Table 

In [ ]:
# explainable_models = ["DecisionTreeClassifier", "LinearTreeClassifier", "RandomForestClassifier", "ElasticNet", "SVC", "LogisticRegression"]
explainable_models = ["DecisionTreeClassifier", "LinearTreeClassifier", "RandomForestClassifier", "ElasticNet", "LogisticRegression"]
black_box_models = ["LinearBoostClassifier", "NeuralNetClassifier", "LGBMClassifier", "XGBClassifier"]

# .drop(columns=["support", "precision", "recall", "metric type"])
all_val_results = avg_val_results.reset_index("metric type")
all_val_results = all_val_results[all_val_results["metric type"] == "weighted avg"].drop(columns=["support", "precision", "recall", "metric type"])
avg_explainable_val_results = all_val_results.loc[explainable_models].sort_values(by="f1-score", ascending=False).reset_index()
black_box_val_results = all_val_results.loc[black_box_models].sort_values(by="f1-score", ascending=False).reset_index()

# all_val_results.loc[["DecisionTreeClassifier", "SVC"]]
explainble_vs_black_box_table_df = pd.concat([avg_explainable_val_results, black_box_val_results], keys=["Explainable", "Black Box"], axis=1)

means = explainble_vs_black_box_table_df.xs('f1-score', axis=1, level=1).mean().to_frame().transpose()

explainble_vs_black_box_table_df.loc[len(explainble_vs_black_box_table_df.index)] = ["Mean", float(means["Explainable"]), "Mean", float(means["Black Box"])]
explainble_vs_black_box_table_df = explainble_vs_black_box_table_df.round(3)
display(explainble_vs_black_box_table_df)
print(explainble_vs_black_box_table_df.to_latex(index=False))

### TPE Explainable vs Black Box TPE Paper Table 

In [ ]:
explainable_models = ["DecisionTreeClassifier", "RandomForestClassifier", "ElasticNet", "SVC", "LogisticRegression"]
black_box_models = ["LinearBoostClassifier", "NeuralNetClassifier", "LGBMClassifier", "XGBClassifier"]

# .drop(columns=["support", "precision", "recall", "metric type"])
all_val_results = avg_tpe_val_results.reset_index("metric type")
all_val_results = all_val_results[all_val_results["metric type"] == "weighted avg"].drop(columns=["support", "precision", "recall", "metric type"])
avg_explainable_val_results = all_val_results.loc[explainable_models].sort_values(by="f1-score", ascending=False).reset_index()
black_box_val_results = all_val_results.loc[black_box_models].sort_values(by="f1-score", ascending=False).reset_index()

# all_val_results.loc[["DecisionTreeClassifier", "SVC"]]
explainble_vs_black_box_table_df = pd.concat([avg_explainable_val_results, black_box_val_results], keys=["Explainable", "Black Box"], axis=1)

means = explainble_vs_black_box_table_df.xs('f1-score', axis=1, level=1).mean().to_frame().transpose()

explainble_vs_black_box_table_df.loc[len(explainble_vs_black_box_table_df.index)] = ["Mean", float(means["Explainable"]), "Mean", float(means["Black Box"])]
explainble_vs_black_box_table_df = explainble_vs_black_box_table_df.round(3)
display(explainble_vs_black_box_table_df)
print(explainble_vs_black_box_table_df.to_latex(index=False))

#### Random Search ANOVA

In [ ]:
from itertools import combinations
from scipy import stats
from tabulate import tabulate
from scipy.stats import shapiro
from scipy.stats import f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

random_search_val_scores = val_results.loc[(slice(None), 'weighted avg'), :].droplevel('metric type').drop(columns=['support', 'precision', 'recall'], axis = 1).reset_index()
random_search_val_scores = random_search_val_scores[~random_search_val_scores["model"].isin(["XGBClassifier", "NeuralNetClassifier"])]
display(random_search_val_scores)
model_scores = {}
p_statistics = []
for model in random_search_val_scores["model"].unique():
#     print(model)
    
    scores = random_search_val_scores.set_index("model").loc[model].values.flatten()
    S, p_shapiro = shapiro(scores)
    p_statistics.append(p_shapiro)
#     print(f"{model} is normall dist: {p_shapiro}")
    model_scores[model] = scores
    

F, p_anova = f_oneway(*model_scores.values()) 
# print(p_anova)

print("All generalization scores are normally dist: ", all(pstat > .05 for pstat in p_statistics))
val_data_tograph = pd.DataFrame(model_scores).melt(var_name="Model", value_name="f1-score")
val_data_tograph["Model"] = val_data_tograph["Model"].str.replace("Classifier", "")


sns.set_palette("pastel")
box_plot = sns.boxplot(x='Model', y='f1-score', data=val_data_tograph, )
plt.text(0.7, .9, f"p={round(p_anova, 4)}", fontsize=12, ha="center", va="center", transform=plt.gca().transAxes)
plt.title("Generalization Set Weighted F1-Score Distributions")
box_plot.set_xticklabels(box_plot.get_xticklabels(), rotation=90)


### TPE Search ANOVA

In [ ]:
tpe_val_scores = tpesearch_val_results.loc[(slice(None), 'weighted avg'), :].droplevel('metric type').drop(columns=['support', 'precision', 'recall'], axis = 1).reset_index()
# tpe_val_scores = tpe_val_scores[~tpe_val_scores["model"].isin(["XGBClassifier", "NeuralNetClassifier"])]
display(tpe_val_scores)
model_scores = {}
p_statistics = []
for model in tpe_val_scores["model"].unique():
#     print(model)
    
    scores = tpe_val_scores.set_index("model").loc[model].values.flatten()
    S, p_shapiro = shapiro(scores)
    p_statistics.append(p_shapiro)
    print(f"{model} is normall dist: {p_shapiro}")
    model_scores[model] = scores
    

F, p_anova = f_oneway(*model_scores.values()) 
# print(p_anova)

print("All generalization scores are normally dist: ", all(pstat > .05 for pstat in p_statistics))
val_data_tograph = pd.DataFrame(model_scores).melt(var_name="Model", value_name="f1-score")
val_data_tograph["Model"] = val_data_tograph["Model"].str.replace("Classifier", "")


sns.set_palette("pastel")
box_plot = sns.boxplot(x='Model', y='f1-score', data=val_data_tograph, )
plt.text(0.7, .9, f"p={round(p_anova, 4)}", fontsize=12, ha="center", va="center", transform=plt.gca().transAxes)
plt.title("Generalization Set Weighted F1-Score Distributions")
box_plot.set_xticklabels(box_plot.get_xticklabels(), rotation=90)

### Get confusion matrices for DF

In [ ]:
from ast import literal_eval
# Get the confusion matrices for each model

def get_confusion_matrices(df, result_dir, model_title="DecisionTree"):
    models = df.index.unique()

    print("models: ", models)
    for model_name in models:
        y_pred = []
        y_val = []

        model_df = df.loc[model_name]
        if isinstance(model_df, pd.DataFrame):
            for index, row in model_df.iterrows():
                y_pred.extend(literal_eval(row["Y Pred"].replace(' ', ",")))
                y_val.extend(literal_eval(row["Y Val"].replace(' ', ",")))

        else:
            y_pred = literal_eval(model_df["Y Pred"].replace(' ', ","))
            y_val = literal_eval(model_df["Y Val"].replace(' ', ","))
        
        seaborn_conf_matrix(confusion_matrix(y_val, y_pred), model_name=model_title, result_dir=result_dir)


# get_confusion_matrices(model_results)

        


In [ ]:
get_confusion_matrices(ensemble_estimator_results.loc["BaggingClassifierElasticNet"], "../figures/confusion_matricies", model_title="Bagged Elastic Net")

### Get Best Scores and Best Params Random Search

In [ ]:
import json
import re
# from ast import eval

def strip_column_transformer(params):
    substrings_to_remove = ["<", "<__", ">", "class"]

    pattern = re.compile("|".join(map(re.escape, substrings_to_remove)))
    params = pattern.sub("", params)

    pattern = r"__main__.NamedFeatureSelector object at 0x\w+"
    params = re.sub(pattern, "'CustomFeatureSelector'", params)
    
    pattern = r"None"
    params = re.sub(pattern, "'None'", params)
    
    print(params)
    params = eval(params)
    
    # print(params, type(params))
    if "selector" in params.keys():
        selector = str(params["selector"])
    else:
        selector = ""

    if "scaler" in params.keys():
        scaler = str(params["scaler"])
    else:
        scaler = ""
    
    if "NamedFeatureSelector" in selector:
        params["selector"] = "Custom Feature Selector"

    if "StandardScaler" in scaler:
        params["scaler"] = "StandardScaler"
    
    if "MinMaxScaler" in scaler:
        params["scaler"] = "MinMaxScaler"

    if "RobustScaler" in scaler:
        params["scaler"] = "RobustScaler"

    # print(params)
    return params

    

In [ ]:
from ast import literal_eval
from sklearn.metrics import f1_score
import pyperclip
# test = estimator_results.loc["DecisionTreeClassifier", ["Y Val", "Y Pred"]]
rs_best_estimator_results = estimator_results.loc[:, ["Y Val", "Y Pred", "Best Params"]]

rs_best_estimator_results["f1-score"] = rs_best_estimator_results.apply(lambda row: f1_score(literal_eval(row["Y Val"].replace(' ', ",")), 
                                                  literal_eval(row["Y Pred"].replace(' ', ",")),
                                                  average="weighted"), axis=1)

rs_best_estimator_results = rs_best_estimator_results.reset_index(
    ).drop(columns=["metric type"]
    ).sort_values(by="f1-score", ascending=False
    ).drop_duplicates(subset=['model'], keep='first')


rs_best_params = rs_best_estimator_results.set_index("model")[["Best Params", "f1-score"]]
rs_best_params["Best Params"] = rs_best_params["Best Params"].apply(strip_column_transformer)
rs_best_params.loc["Mean", "f1-score"] = rs_best_params["f1-score"].mean()
display(rs_best_params)

rs_best_params_latex_table = rs_best_params.round(3).to_latex()
print(rs_best_params_latex_table)



### Best Hyperparams Best Score TPE

In [ ]:
tpe_best_estimator_results = tpe_estimator_results.loc[:, ["Y Val", "Y Pred", "Best Params"]]

tpe_best_estimator_results["f1-score"] = tpe_best_estimator_results.apply(lambda row: f1_score(literal_eval(row["Y Val"].replace(' ', ",")), 
                                                  literal_eval(row["Y Pred"].replace(' ', ",")),
                                                  average="weighted"), axis=1)

tpe_best_estimator_results = tpe_best_estimator_results.reset_index(
    ).drop(columns=["metric type"]
    ).sort_values(by="f1-score", ascending=False
    ).drop_duplicates(subset=['model'], keep='first')


tpe_best_params = tpe_best_estimator_results.set_index("model")[["Best Params", "f1-score"]]
tpe_best_params["Best Params"] = tpe_best_params["Best Params"].apply(strip_column_transformer)
tpe_best_params.loc["Mean", "f1-score"] = tpe_best_params["f1-score"].mean()
display(tpe_best_params)

tpe_best_params_latex_table = tpe_best_params.round(3).to_latex()
print(tpe_best_params_latex_table)

### SVC Explainability

In [ ]:
from sklearn.metrics import f1_score
from ast import literal_eval
from sklearn.pipeline import make_pipeline
svm_results = tpe_estimator_results.loc["SVC"]
svm_results["f1-score"] = svm_results.apply(lambda row: f1_score(literal_eval(row["Y Val"].replace(' ', ",")), 
                                                  literal_eval(row["Y Pred"].replace(' ', ",")),
                                                  average="weighted"), axis=1)

best_svm = svm_results.sort_values(by="f1-score", ascending=False).iloc[0].to_frame().transpose()
test = best_svm["Best Estimator"].values[0]


pipeline = Pipeline([
            ('scaler', myStandardScaler),
            ('selector', NamedFeatureSelector(["Delta_MT", "Delta_AE", "cvRT_HR", "Corrective_HR", "RT_V", "age", "NHL"])),
            ('model',
                 SVC(C=1.551, class_weight='balanced', gamma=51,
                     kernel='sigmoid', random_state=42))

        ])

X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="previous_concussions")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42, stratify=y)
# cv = get_cross_validation(X_train, y_train, n_splits=10, type="stratified", random_state=random_state)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_val)

print(f1_score(y_val, y_pred))


# pca = PCA(n_components=.8, random_state=SEED)
# pipe = Pipeline([('scaler', myStandardScaler),
#             ('selector', NamedFeatureSelector(["Delta_MT", "Delta_AE", "cvRT_HR", "Corrective_HR", "RT_V", "age", "NHL"])),
#             ('pca', pca)])
# Xt = pipe.fit_transform(X)
# plot = plt.scatter(Xt[:,0], Xt[:,1], c=y)
# plt.legend(handles=plot.legend_elements()[0], labels=['No Concussion', 'Concussion'])
# plt.show()


### Graph Decision Boundary

In [ ]:
def show_decision_boundary(clf, X, y):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))

    # Reshape the meshgrid points for prediction
    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    
    # Make predictions on meshgrid points


#     f1Score = f1_score(y, Z)
#     print(f1Score)

    # Plot the decision boundary
    start_index = 30
    end_index = start_index + 24
    generalizeX = X[start_index : end_index, 0]
    generalizeY = X[start_index : end_index, 1]
    np.random.seed(SEED)
    generalizeT = y[start_index : end_index]
    
    Z = clf.predict(mesh_points)
    f1Score = f1_score(generalizeT, Z[start_index : end_index])
    
    print(f1Score)
    Z = Z.reshape(xx.shape)
    

    plt.contourf(xx, yy, Z, alpha=0.5)
    plt.scatter(generalizeX, generalizeY, c=generalizeT, edgecolors='k')
    plt.text(0.7, .9, f"f1 = {round(f1Score, 3)}", fontsize=12, ha="center", va="center", transform=plt.gca().transAxes)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('Decision Boundary of SVM (PCA-reduced space)')
    plt.show()

pca = PCA(n_components=2, random_state=42)
pca_pipe = Pipeline([('scaler', StandardScaler()),
            ('pca', pca)])

X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="previous_concussions")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42, stratify=y)


# svm_model = pipeline.named_steps['model']
X_train = pca_pipe.fit_transform(X_train)

svm_model = SVC(C=1.551, class_weight='balanced', gamma=51, kernel='sigmoid', random_state=42)


svm_model.fit(X_train, y_train)


show_decision_boundary(svm_model, X_train, y_train)

In [ ]:

from sklearn.svm import SVC
import numpy as np
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


pca = PCA(n_components=3, random_state=42)
pca_pipe = Pipeline([('scaler', StandardScaler()),
                     ('pca', pca)])

X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="previous_concussions")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42, stratify=y)

X_train = pca_pipe.fit_transform(X_train)

svm_model = SVC(C=1.551, class_weight='balanced', gamma=51, kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Define the equation of the separating plane
w = svm_model.coef_[0]
b = svm_model.intercept_[0]
xx, yy = np.meshgrid(np.linspace(-5, 5, 30), np.linspace(-5, 5, 30))
zz = (-w[0] * xx - w[1] * yy - b) / w[2]

# Create a 3D scatter plot
fig = px.scatter_3d(x=X_train[:, 0], y=X_train[:, 1], z=X_train[:, 2], color=y_train)
fig.add_surface(x=xx, y=yy, z=zz, showscale=False, opacity=0.8)

fig.update_layout(scene=dict(
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    zaxis_title='Principal Component 3'
))

fig.show()


In [ ]:
from sklearn.svm import SVC
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from mpl_toolkits.mplot3d import Axes3D

iris = datasets.load_iris()
X = iris.data[:, :3]  # we only take the first three features.
Y = iris.target

#make it binary classification problem
X = X[np.logical_or(Y==0,Y==1)]
Y = Y[np.logical_or(Y==0,Y==1)]

display(X[:5])
display(Y[:5])

model = svm.SVC(kernel='linear')
clf = model.fit(X, Y)

# The equation of the separating plane is given by all x so that np.dot(svc.coef_[0], x) + b = 0.
# Solve for w3 (z)
z = lambda x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]

tmp = np.linspace(-5,5,30)
x,y = np.meshgrid(tmp,tmp)

fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')
ax.plot3D(X[Y==0,0], X[Y==0,1], X[Y==0,2],'ob')
# ax.plot3D(X[Y==1,0], X[Y==1,1], X[Y==1,2],'sr')
ax.plot_surface(x, y, z(x,y))
ax.view_init(30, 60)
plt.show()

### SVM Feature Importance Plot

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(pipeline, X_val, y_val, n_repeats=10, random_state=42)
feature_importances = result.importances_mean
feature_names = ["Delta_MT", "Delta_AE", "cvRT_HR", "Corrective_HR", "RT_V", "age", "NHL"]
feature_importances = [i for i in feature_importances if i != 0]
print(feature_names)
print(feature_importances)
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df

### Feature Bar Chart

In [ ]:
importance_df = importance_df.sort_values('Importance', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(y='Importance', x='Feature', data=importance_df, palette="blend:#7AB,#EDA")
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance (Permutation Importance)')
plt.show()


In [ ]:
X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="previous_concussions")
pca = PCA(n_components=.8, random_state=SEED)
pipe = Pipeline([('scaler', StandardScaler()), ('pca', pca)])
Xt = pipe.fit_transform(X)
plot = plt.scatter(Xt[:,0], Xt[:,1], c=y)
plt.legend(handles=plot.legend_elements()[0], labels=['No Concussion', 'Concussion'])
plt.show()

## NOTES / Scratch

### Optuna Scratch Work

In [ ]:
def objective(trial):
    # -- Instantiate scaler
    # (a) List scalers to chose from
    scalers = trial.suggest_categorical("scalers", ['minmax', 'standard', 'robust'])

    # (b) Define your scalers
    if scalers == "minmax":
        scaler = MinMaxScaler()
    elif scalers == "standard":
        scaler = StandardScaler()
    else:
        scaler = RobustScaler()

    # -- Instantiate dimensionality reduction
     # (a) List all dimensionality reduction options
    dim_red = trial.suggest_categorical("dim_red", ["PCA", None])

    # (b) Define the PCA algorithm and its hyperparameters
    if dim_red == "PCA":
        pca_n_components=trial.suggest_int("pca_n_components", 2, 30) # suggest an integer from 2 to 30
        dimen_red_algorithm=PCA(n_components=pca_n_components)
    # (c) No dimensionality reduction option
    else:
        dimen_red_algorithm='passthrough'

    # -- Instantiate estimator model
    knn_n_neighbors=trial.suggest_int("knn_n_neighbors", 1, 19, 2) # suggest an integer from 1 to with step 2
    knn_metric=trial.suggest_categorical("knn_metric", ['euclidean', 'manhattan', 'minkowski'])
    knn_weights=trial.suggest_categorical("knn_weights", ['uniform', 'distance'])

    estimator=KNeighborsClassifier(n_neighbors=knn_n_neighbors, metric=knn_metric, weights=knn_weights)

    # -- Make a pipeline
    pipeline = make_pipeline(scaler, dimen_red_algorithm, estimator)

    # -- Evaluate the score by cross-validation
    score = cross_val_score(pipeline, X, y, scoring='f1')
    f1 = score.mean() # calculate the mean of scores

In [ ]:

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="NHL")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42, stratify=y)
cv = get_cross_validation(X_train, y_train, n_splits=10, type="stratified", random_state=42)

# Create a study name:
study_name = 'experiment-C'

# Store in DB:
study = optuna.create_study(study_name="test-DT",)

#Step 1. Define an objective function to be maximized.
def objective(trial):

    scalers = trial.suggest_categorical("scalers", ['minmax', 'standard', 'robust'])

    # (b) Define your scalers
    if scalers == "minmax":
        scaler = MinMaxScaler()
    elif scalers == "standard":
        scaler = StandardScaler()
    else:
        scaler = RobustScaler()
        
    classifier_name = trial.suggest_categorical("classifier", ["RandomForest"])
    
    # Step 2. Setup values for the hyperparameters:
    if classifier_name == 'LogReg':
        logreg_c = trial.suggest_float("logreg_c", 1e-10, 1e10, log=True)
        classifier_obj = linear_model.LogisticRegression(C=logreg_c)
    else:
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators
        )

    pipeline = make_pipeline(scaler, classifier_obj)
    # Step 3: Scoring method:
    score = cross_val_score(pipeline, X_train, y_train, n_jobs=-1, cv=cv, scoring=make_scorer(f1_score, average="weighted"))
    weighted_f1 = score.mean()
    return weighted_f1

    # Step 4: Running it
#     study = joblib.load('experiments.pkl')
# study.optimize(objective, n_trials=3)
#     joblib.dump(study, 'experiments.pkl')
# study.trials_dataframe()

In [ ]:
def objective(trial):
    X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="NHL")
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42, stratify=y)
#     cv = get_cross_validation(X_train, y_train, n_splits=10, type="stratified", random_state=42)

    model = DecisionTreeClassifier(random_state=SEED)
    pipe = Pipeline([("scaler", myStandardScaler), ("model", model)])
    
#     max_depth = 
#     solver = trial.suggest_categorical("solver", ("lbfgs", "saga"))
    
    params = {
        "model__max_depth" : trial.suggest_int("model__max_depth", 1, 20, log=True),
        "model__class_weight" : "balanced"
    }
       
    pipe.set_params(**params)
    pipe.fit(X_train, y_train)
   
    y_pred_train = pipe.predict(X_train)
    train_f1 = f1_score(y_train, y_pred_train, average='weighted')
#     print(X_val, y_val)
    y_pred_val = pipe.predict(X_val)
    val_f1 = f1_score(y_val, y_pred_val, average='weighted')

    print(f'Train: {train_f1} Val: {val_f1}')
    return val_f1, train_f1 - val_f1

# study = optuna.create_study(directions=["maximize", "minimize"])
# study.optimize(objective, n_trials=10)

# best_trials = study.best_trials
# print("Best Parameters: ", best_trials)

In [ ]:
X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="NHL")
    


def objective(trial):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42, stratify=y)

    cv = get_cross_validation(X_train, y_train, n_splits=10, type="stratified", random_state=42)


#     C = trial.suggest_float("C", 1e-7, 10.0, log=True)
#     solver = trial.suggest_categorical("solver", ("lbfgs", "saga"))

#     clf = LogisticRegression(C=C, solver=solver)
#     clf.fit(X_train, y_train)
#     val_accuracy = clf.score(X_test, y_test)
    
    param_distributions = {
        "model__max_depth" : optuna.distributions.IntDistribution(2, 10)
    }

    
#    score = cross_val_score(pipeline, X_train, y_train, n_jobs=-1, cv=cv, scoring=make_scorer(f1_score, average="weighted"))
    optuna_search = optuna.integration.OptunaSearchCV(pipe, param_distributions, cv=cv, refit=True, scoring=make_scorer(f1_score, average="weighted"))
    optuna_search.fit(X_train, y_train)
    
    val_accuracy = 

    return val_accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

In [ ]:
X, y = create_dataset(clean_raw_data(f"../{DATA_DIR}Brdi_db_march.xlsx"), target_col="NHL")
    
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42, stratify=y)

cv = get_cross_validation(X_train, y_train, n_splits=10, type="stratified", random_state=42)

model = DecisionTreeClassifier(random_state=SEED)

param_distributions = {
    "model__max_depth" : optuna.distributions.IntDistribution(2, 10)
}

pipe = Pipeline([("scaler", myStandardScaler), ("model", model)])
optuna_search = optuna.integration.OptunaSearchCV(pipe, param_distributions)

optuna_search.fit(X_train, y_train)

display(optuna_search.trials_dataframe)
# y_pred = optuna_search.predict(X_val)

In [ ]:
def df_style(val):
    return "font-weight: bold"

summary = pd.DataFrame([[0, 0, 0, 0, 0], [1620, 203, 392, 651, 2236], [1620, 203, 392, 651, 2236]],
                       index=["None", "Total", "Total"])

summary = summary.reset_index()
display(summary)
last_row = summary.index[summary["index"] == "Total"]
print(last_row)
# # get a handle on the row that starts with `"Total"`, i.e., the last row here
# last_row = pd.IndexSlice[summary.index[summary.index == "Total"], :]
# # and apply styling to it via the `subset` arg; first arg is styler function above
summaryStyled = summary.style.applymap(df_style, subset=(slice(0, len(), 2), ["f1-score"]))

display(summaryStyled)

In [ ]:
def df_style(val):
    return "font-weight: bold"


# tpesearch_val_results = tpesearch_val_results.style.applymap(df_style, subset=(slice(0, len(tpesearch_val_results), 5), ["f1-score"]))
avg_tpe_val_results = tpesearch_val_results.groupby(tpesearch_val_results.index.names, group_keys=False).mean()
avg_tpe_val_results = sort_by_metric(avg_tpe_val_results, METRIC_TYPE, METRIC)
# avg_tpe_val_results = avg_tpe_val_results.style.applymap(df_style, subset=(slice(0, len(test), 4), ["f1-score", "recall" ,"precision"])).data.set_index(["model", "metric type"])
avg_tpe_val_results.to_excel("../mean_results/avg_tpe_train_results_80_20.xlsx", index=True)
# avg_tpe_val_results = 
avg_tpe_val_results